## Tractography : probabilistic tracking

Considering that the SNR of the measurement is pretty low, and that the information about directions of fascicles that we get in every voxel is not that accurate, we might want to consider this information with a grain of salt. One way to do that is to treat the directions provided by the [reconstruction](DTI.ipynb) [algorithms](SFM.ipynb) as the modes of a probability distribution and, in each iteration of our tracking algorithm, sample from this probability distribution. This also has the benefit that it allows us to track through locations in which the directional signal to the reconstruction algorithms is not as clear, due to SNR issues, or due to spatial and directional resolution of measurement. This approach, pioneered by Behrens et al [1] is called "probabilistic tractography". 

[1] Behrens, T E J, H Johansen-Berg, M W Woolrich, S M Smith, C A M Wheeler-Kingshott, P A Boulby, G J Barker, et al. 2003. “Non-Invasive Mapping of Connections between Human Thalamus and Cortex Using Diffusion Imaging.” Nature Neuroscience 6: 750–57.

In [2]:
import os
import os.path as op
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt
%matplotlib inline

import dipy.core.gradients as grad
from dipy.data import get_sphere
from dipy.reconst.csdeconv import auto_response
from dipy.reconst import sfm, dti
from dipy.io.trackvis import save_trk
from dipy.reconst.peaks import peaks_from_model
from dipy.data import small_sphere
from dipy.io.trackvis import save_trk
from dipy.direction import ProbabilisticDirectionGetter
from dipy.tracking.local import ThresholdTissueClassifier
from dipy.tracking.local import LocalTracking

from IPython.display import display, Image

In [3]:
from glob import glob
from nipype.interfaces.freesurfer.model import Binarize
from nipype.interfaces.ants.registration import Registration

In [4]:
if not 'FREESURFER_HOME' in os.environ:
    os.environ['FREESURFER_HOME'] = '/Applications/freesurfer'
    os.environ['SUBJECTS_DIR'] = '/Applications/freesurfer/subjects'

In [5]:
os.getenv('FREESURFER_HOME')

'/Applications/freesurfer'

In [6]:
sub = 'GAB_ISN_012'

data_dir = op.join('/Users/kevinsitek/om','om/user/satra/projects/isn', 'data')

project_dir = op.join('/Users/kevinsitek/om','om/user/ksitek/isn')
tracula_dir = op.join(project_dir, 'tracula')

In [10]:
# mount the data drive if not already
if ~os.path.isdir(project_dir):
    os.system("sshfs ~/om ksitek@openmind7.mit.edu:/ -o defer_permissions")

In [7]:
dwi_img = op.join(data_dir, sub, 'dmri/preproc','dwi_7p5k_001_disco.nii.gz')

dwi_ni = nib.load(dwi_img)
#LV1_ni = nib.load(op.join('data', 'SUB1_LV1.nii.gz'))
#labels_ni = nib.load(op.join('data', 'SUB1_aparc-reduced.nii.gz'))



In [8]:
data = dwi_ni.get_data()
affine = dwi_ni.get_affine()

#LV1_data = LV1_ni.get_data()
#labels = labels_ni.get_data()

In [9]:
bvecs = op.join(data_dir,sub,'dmri','dwi_7p5k_001.bvecs')
bvals = op.join(data_dir,sub,'dmri','dwi_7p5k_001.bvals')

In [10]:
gtab = grad.gradient_table(bvals, bvecs)

In [11]:
# create white matter mask (from our Eddy_tracula_csd.py script)
fmask1 = op.abspath(glob(op.join(tracula_dir, 'dwi_7p5k_001/%s/dlabel/diff/aparc+aseg_mask.bbr.nii.gz' % sub))[0])
fmask2 = op.abspath(glob(op.join(tracula_dir, 'dwi_7p5k_001/%s/dlabel/diff/notventricles.bbr.nii.gz' % sub))[0])
mask = (nib.load(fmask1).get_data() > 0.5) * nib.load(fmask2).get_data()

In [12]:
from nipype.interfaces.freesurfer import Info

Info.subjectsdir()

'/Applications/freesurfer/subjects'

In [13]:
# transform the freesurfer segmentation image to diffusion space
aparc = op.join('/Users/kevinsitek/om','om/user/satra/projects/isn','fsdata/GAB_ISN_012/mri/aparc+aseg.mgz')

# first, turn the segmentation label file into a volume
from nipype.interfaces.freesurfer.model import Label2Vol
label2vol = Label2Vol(seg_file=aparc, 
                      template_file=op.join(tracula_dir,'dwi_7p5k_001/',sub,'dmri/lowb_brain.nii.gz'), 
                      reg_file = op.join(tracula_dir,'dwi_7p5k_001/',sub,'dmri/xfms/anat2diff.bbr.mat'), 
                      vol_label_file='aparc+aseg_vol.nii.gz')
label2vol.run()

'''
from nipype.interfaces.freesurfer.preprocess import MRIConvert
mc = MRIConvert()
mc.inputs.in_file = aparc
mc.inputs.out_file = 'aparc+aseg.nii.gz'
mc.run()
'''

from nipype.interfaces import fsl
flt = fsl.FLIRT(bins=640, cost_func='mutualinfo')
flt.inputs.in_file = 'aparc+aseg_vol.nii.gz'
flt.inputs.reference = dwi_img
flt.inputs.output_type = "NIFTI_GZ"
flt.run()

INFO:interface:stdout 2016-11-07T11:11:52.988867:Number of labels: 0
INFO:interface:stdout 2016-11-07T11:11:52.988867:Annot File:      (null)
INFO:interface:stdout 2016-11-07T11:11:52.988867:Template Volume: /Users/kevinsitek/om/om/user/ksitek/isn/tracula/dwi_7p5k_001/GAB_ISN_012/dmri/lowb_brain.nii.gz
INFO:interface:stdout 2016-11-07T11:11:52.988867:Outut Volume: aparc+aseg_vol.nii.gz
INFO:interface:stdout 2016-11-07T11:11:52.988867:Registration File: /Users/kevinsitek/om/om/user/ksitek/isn/tracula/dwi_7p5k_001/GAB_ISN_012/dmri/xfms/anat2diff.bbr.mat
INFO:interface:stdout 2016-11-07T11:11:52.988867:Fill Threshold: 0
INFO:interface:stdout 2016-11-07T11:11:52.988867:Label Vox Vol:  1
INFO:interface:stdout 2016-11-07T11:11:52.988867:ProjType:       (null)
INFO:interface:stdout 2016-11-07T11:11:52.988867:ProjTypeId:     0
INFO:interface:stdout 2016-11-07T11:11:52.988867:ProjStart:      0
INFO:interface:stdout 2016-11-07T11:11:52.988867:ProjStop:       0
INFO:interface:stdout 2016-11-07T11

RuntimeError: Command:
mri_label2vol --reg /Users/kevinsitek/om/om/user/ksitek/isn/tracula/dwi_7p5k_001/GAB_ISN_012/dmri/xfms/anat2diff.bbr.mat --seg /Users/kevinsitek/om/om/user/satra/projects/isn/fsdata/GAB_ISN_012/mri/aparc+aseg.mgz --temp /Users/kevinsitek/om/om/user/ksitek/isn/tracula/dwi_7p5k_001/GAB_ISN_012/dmri/lowb_brain.nii.gz --o aparc+aseg_vol.nii.gz
Standard output:
Number of labels: 0
Annot File:      (null)
Template Volume: /Users/kevinsitek/om/om/user/ksitek/isn/tracula/dwi_7p5k_001/GAB_ISN_012/dmri/lowb_brain.nii.gz
Outut Volume: aparc+aseg_vol.nii.gz
Registration File: /Users/kevinsitek/om/om/user/ksitek/isn/tracula/dwi_7p5k_001/GAB_ISN_012/dmri/xfms/anat2diff.bbr.mat
Fill Threshold: 0
Label Vox Vol:  1
ProjType:       (null)
ProjTypeId:     0
ProjStart:      0
ProjStop:       0
ProjDelta:      0.1
Subject:  (null)
Hemi:     (null)
UseNewASeg2Vol:  1
DoLabelStatVol  0
LabelCodeOffset  0
setenv SUBJECTS_DIR /Applications/freesurfer/subjects
$Id: mri_label2vol.c,v 1.34.

In [47]:
# create ROI (1022 = ctx-lh-postcentral)
binvol = Binarize(in_file='aparc+aseg.nii.gz', binary_file='lh-postcentral_fs.nii')
binvol.inputs.match = [1022]
binvol.run()

INFO:interface:stdout 2016-10-07T14:09:56.408414:
INFO:interface:stdout 2016-10-07T14:09:56.408414:$Id: mri_binarize.c,v 1.26.2.1 2011/04/08 15:40:50 greve Exp $
INFO:interface:stdout 2016-10-07T14:09:56.408414:cwd /Users/kevinsitek/Dropbox (MIT)/projects/visual-white-matter
INFO:interface:stdout 2016-10-07T14:09:56.408414:cmdline mri_binarize --o lh-postcentral_fs.nii --i aparc+aseg.nii.gz --match 1022 
INFO:interface:stdout 2016-10-07T14:09:56.408414:sysname  Darwin
INFO:interface:stdout 2016-10-07T14:09:56.408414:hostname dhcp-18-111-77-21.dyn.mit.edu
INFO:interface:stdout 2016-10-07T14:09:56.408414:machine  x86_64
INFO:interface:stdout 2016-10-07T14:09:56.408414:user     kevinsitek
INFO:interface:stdout 2016-10-07T14:09:56.408414:
INFO:interface:stdout 2016-10-07T14:09:56.408414:input      aparc+aseg.nii.gz
INFO:interface:stdout 2016-10-07T14:09:56.408414:frame      0
INFO:interface:stdout 2016-10-07T14:09:56.408414:nErode3d   0
INFO:interface:stdout 2016-10-07T14:09:56.408414:nEro

In [52]:
from nipype.interfaces.freesurfer import ApplyVolTransform
applyreg = ApplyVolTransform()
applyreg.inputs.source_file = 'lh-postcentral_fs.nii'
applyreg.inputs.target_file = op.join(tracula_dir,'dwi_7p5k_001/',sub,'dmri/lowb_brain.nii.gz')
applyreg.inputs.fsl_reg_file = op.join(tracula_dir,'dwi_7p5k_001/',sub,'dmri/xfms/anat2diff.bbr.mat')
applyreg.inputs.transformed_file = 'lh-postcentral_diff.nii'
applyreg.run()

INFO:interface:stdout 2016-10-07T14:14:34.415464:movvol lh-postcentral_fs.nii
INFO:interface:stdout 2016-10-07T14:14:34.415464:targvol /Users/kevinsitek/om/om/user/ksitek/isn/tracula/dwi_7p5k_001/GAB_ISN_012/dmri/lowb_brain.nii.gz
INFO:interface:stdout 2016-10-07T14:14:34.415464:outvol lh-postcentral_diff.nii
INFO:interface:stdout 2016-10-07T14:14:34.415464:invert 0
INFO:interface:stdout 2016-10-07T14:14:34.415464:tal    0
INFO:interface:stdout 2016-10-07T14:14:34.415464:talres 2
INFO:interface:stdout 2016-10-07T14:14:34.415464:regheader 0
INFO:interface:stdout 2016-10-07T14:14:34.415464:noresample 0
INFO:interface:stdout 2016-10-07T14:14:34.415464:interp  trilinear (1)
INFO:interface:stdout 2016-10-07T14:14:34.415464:precision  float (3)
INFO:interface:stdout 2016-10-07T14:14:34.415464:Gdiag_no  -1
INFO:interface:stdout 2016-10-07T14:14:34.415464:Synth      0
INFO:interface:stdout 2016-10-07T14:14:34.415464:SynthSeed  1476505192
INFO:interface:stdout 2016-10-07T14:14:34.415464:
INFO:i

In [14]:
seed_ni = nib.load('lh-postcentral_fs.nii')
seed_data = seed_ni.get_data()

In [15]:
#white_matter =  (labels == 1) | (labels == 2)
#V1 = (LV1_data == 1)

In [16]:
white_matter =  mask
seed = (seed_data == 1)

In [17]:
response, ratio = auto_response(gtab, data, roi_radius=10, fa_thr=0.7)

/Users/kevinsitek/miniconda/lib/python2.7/site-packages/dipy/reconst/csdeconv.py:818: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  roi = data[ci - w: ci + w, cj - w: cj + w, ck - w: ck + w]


In [18]:
sphere = get_sphere()
sf_model = sfm.SparseFascicleModel(gtab, sphere=sphere,
                                   l1_ratio=0.5, alpha=0.001,
                                   response=response[0])

We're going to fit the model in the entire white matter in the following cell, so this might take a little while to run:

In [21]:
sf_fit = sf_model.fit(data, mask=white_matter)

/Users/kevinsitek/miniconda/lib/python2.7/site-packages/dipy/reconst/sfm.py:385: RuntimeWarning: divide by zero encountered in divide
  flat_S = flat_data[..., ~self.gtab.b0s_mask] / flat_S0[..., None]
/Users/kevinsitek/miniconda/lib/python2.7/site-packages/dipy/reconst/sfm.py:83: RuntimeWarning: divide by zero encountered in divide
  to_fit = data_no_b0 / s0[..., None]
/Users/kevinsitek/miniconda/lib/python2.7/site-packages/sklearn/linear_model/coordinate_descent.py:466: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations
  ConvergenceWarning)
/Users/kevinsitek/miniconda/lib/python2.7/site-packages/sklearn/linear_model/coordinate_descent.py:466: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations
  ConvergenceWarning)
/Users/kevinsitek/miniconda/lib/python2.7/site-packages/sklearn/linear_model/coordinate_descent.py:466: ConvergenceWarning: Objective did not converge. You might want to incre

In [30]:
# this took forever to generate, so let's pickle it
import cPickle as pickle

pickle.dump(sf_fit, open( "sf_fit.p", "wb" ))

In [ ]:
# so then if it's already been run, load it
import cPickle as pickle

pickle.load( open( "sf_fit.p", "rb" ) )

Probabilistic tracking requires a mechanism to sample directions from the distribution of directions provided by the model. 

In this case

In [22]:
fodf = sf_fit.odf(small_sphere)

prob_dg = ProbabilisticDirectionGetter.from_pmf(fodf, max_angle=30., sphere=small_sphere)

EDIT: adding contextual enhancement

**not yet run - need to make sure all the inputs work with this model type**

*is contextual enhancement section separate from sharpening?*

In [ ]:
fodf = sf_fit.odf(small_sphere)

# KRS 2016-10-05: adding contextual enhancement via Sharpening Deconvolution Transform to sharpen the ODF field.
# Sharpen via the Sharpening Deconvolution Transform
from dipy.reconst.csdeconv import odf_sh_to_sharp
csd_shm_enh_sharp = odf_sh_to_sharp(csd_shm_enh, sphere,  sh_order=8, lambda_=0.1)

# Convert raw and enhanced data to discrete form
csd_sf_orig = sh_to_sf(csd_shm_orig, sphere, sh_order=8)
csd_sf_noisy = sh_to_sf(csd_shm_noisy, sphere, sh_order=8)
csd_sf_enh = sh_to_sf(csd_shm_enh, sphere, sh_order=8)
csd_sf_enh_sharp = sh_to_sf(csd_shm_enh_sharp, sphere, sh_order=8)

# Normalize the sharpened ODFs
csd_sf_enh_sharp = csd_sf_enh_sharp * np.amax(csd_sf_orig)/np.amax(csd_sf_enh_sharp)

prob_dg = ProbabilisticDirectionGetter.from_pmf(fodf, max_angle=30., sphere=small_sphere)

As in the determinstic tracking, we need an object that will terminate tracking based on tissue classification. Again, we will use the `TheresholdTissueClassifier` with FA>0.1

In [23]:
classifier = ThresholdTissueClassifier(white_matter.astype(float), 0.0)

We'll seed the tracking at the pre-defined V1 ROI. We distribute plenty of seeds around V1: 8 in every voxel in the ROI, distributed as `[2, 2, 2]`, that is at a sampling rate of 2x2x2 in each voxel, along each dimension (x/y/z).

In [24]:
import scipy.ndimage as ndi

In [54]:
np.unique(seed.astype(int))

memmap([0, 1])

Our ROI resides in the gray matter, so we need to dilate it a little bit into the white matter. This gives as an ROI that contains not only gray matter voxels, but also the white-matter voxels that are adjacent to this part of cortex

In [55]:
seed_smoothed = ndi.gaussian_filter(seed.astype(float), sigma=0.25).astype(bool)

In [56]:
from dipy.tracking import utils
seeds = utils.seeds_from_mask(seed_smoothed, density=[2, 2, 2], affine=affine)

Finally, we are ready to perform the tracking itself. Tracking will be based on all the elements that we have defined so far. In the course of tracking, we will take steps of 0.5 mm. At the same time, we retain only tracks that have more than 10 nodes in them (length of >5 mm).

In [19]:
# main disco 'seems to contain shearing' - so do all the others in the data/.../preproc :

#dwi_img = op.join(data_dir, sub, 'dmri/','dwi_7p5k_001.nii.gz')
dwi_img = op.join(tracula_dir, 'dwi_7p5k_001', sub, 'dmri/dwi.nii.gz')

dwi_ni = nib.load(dwi_img)
affine = dwi_ni.get_affine()

streamlines = LocalTracking(prob_dg, classifier, seeds, affine, step_size=.5)

NameError: name 'prob_dg' is not defined

In [1]:
affine

NameError: name 'affine' is not defined

In [ ]:
len_th = 10
streamlines = [s for s in streamlines if s.shape[0]>len_th]

In [ ]:
t1_img = op.join(data_dir, sub, 'anatomy/T1_001.nii.gz')

flt = fsl.FLIRT(bins=640, cost_func='mutualinfo')
flt.inputs.in_file = t1_img
flt.inputs.reference = dwi_img
flt.inputs.output_type = "NIFTI_GZ"
flt.inputs.out_file = 'T1_diffspace.nii.gz'
flt.run()

In [ ]:
from dipy.viz import fvtk
from dipy.viz.colormap import line_colors
from dipy.data import read_stanford_t1
from dipy.tracking.utils import move_streamlines
from numpy.linalg import inv
t1 = nib.load('T1_diffspace.nii.gz')
t1_data = t1.get_data()
t1_aff = t1.get_affine()
color = line_colors(streamlines)

In [ ]:
streamlines_actor = fvtk.streamtube(
                    list(move_streamlines(streamlines, inv(t1_aff))),
                                    line_colors(streamlines))

In [ ]:
vol_actor = fvtk.slicer(t1_data)
vol_actor.display_extent(0, t1_data.shape[0]-1, 0, t1_data.shape[1]-1, 25, 25)

ren = fvtk.ren()
fvtk.add(ren, streamlines_actor)
fvtk.add(ren, vol_actor)
fvtk.camera(ren, viewup=(1,0,1), verbose=False)
fvtk.record(ren, out_path='prob-track.png', size=(600,600))

In [ ]:
fvtk.show(ren)

In [ ]:
display(Image(filename='prob-track.png'))

In [ ]:
save_trk("prob-track.trk", streamlines, affine, data.shape[:3])